
  $G_i + P \quad[k^{b_P}_i]<->[k^{u_P}_i] \quad  G_i:P \\
    G_i:P \quad  --> [k_i^{tx}] \quad  G_i + P + T_i \\
    T_i + R \quad  [k^{b_R}_i]<->[k^{u_R}_i]\quad   T_i:R \\
    T_i:R  \quad -->[k_i^{tl}]\quad T_i + R + X_i \\
    T_i + E \quad  [k^{b_E}_i]<->[k^{u_E}_i] \quad  T_i:E \\
    T_i:E \quad-->[\delta_i]\quad E \\
    X \quad-->[d]\quad $

In [293]:

# Reduced model 1
def model1(t,x, params_values):
    k_bp, k_up, k_tx, k_br, k_ur, k_tl,k_be, k_ue, d_i, d, E_tot, P_tot, R_tot, G = params_values
    T = x[0]
    X = x[1]
    y = np.zeros(len(x))
    y[0] = -R_tot*T*k_br*(k_tl + k_ur)/(T*k_br + k_tl + k_ur) + k_tx*(-P_tot*(k_tx + k_up)/(G*k_bp + k_tx + k_up) + P_tot) + (k_tl + k_ur)*(-R_tot*(k_tl + k_ur)/(T*k_br + k_tl + k_ur) + R_tot)
    y[1] = -X*d + k_tl*(-R_tot*(k_tl + k_ur)/(T*k_br + k_tl + k_ur) + R_tot)
    return y

# Reduced model 2
def model2(t,x, params_values):
    k_bp, k_up, k_tx, k_br, k_ur, k_tl,k_be, k_ue, d_i, d, E_tot, P_tot, R_tot, G = params_values
    y = np.zeros(len(x))
    T = x[0]
    R = x[1]
    X = x[2]
    y[0] = -R*T*k_br + k_tx*(-P_tot*(k_tx + k_up)/(G*k_bp + k_tx + k_up) + P_tot) + (-R + R_tot)*(k_tl + k_ur)
    y[1] = -R*T*k_br + (-R + R_tot)*(k_tl + k_ur)
    y[2] = -X*d + k_tl*(-R + R_tot)
    return y

# Full model 
def full_model(t,x, params_values):
    k_bp, k_up, k_tx, k_br, k_ur, k_tl,k_be, k_ue, d_i, d, E_tot, P_tot, R_tot, G = params_values
    x0, x1, x2, x3, x4, x5, x6, x7 = list(x)
    f0 = (k_up + k_tx) * x1 - k_bp * G * x0
    f1 = k_bp * G * x0 - (k_up + k_tx)*x1
    f2 = k_tx * x1 + (k_ur + k_tl) * x4 - k_br * x2 * x3
    f3 = (k_ur + k_tl) * x4 - k_br * x2 * x3
    f4 = k_br * x2 * x3 - (k_ur + k_tl) * x4
    f5 = (k_ue + d_i) * x6 - k_be * x2 * x5
    f6 = k_be * x2 * x5 - (k_ue + d_i) * x6
    f7 = k_tl * x4 - d * x7
    y = np.array([f0,f1,f2,f3,f4,f5,f6,f7])
    return y

# Full model with conservation laws
def f_cons(t,x, params_values):
    k_bp, k_up, k_tx, k_br, k_ur, k_tl,k_be, k_ue, d_i, d, E_tot, P_tot, R_tot, G = params_values
    P, T, R, E, X = x
#     G = 0.01
    f0 = -G*P*k_bp + (-P + P_tot)*(k_tx + k_up)
    f1 = -R*T*k_br + k_tx*(-P + P_tot) + (-R + R_tot)*(k_tl + k_ur)
    f2 = -R*T*k_br + (-R + R_tot)*(k_tl + k_ur)
    f3 = -E*T*k_be + (-E + E_tot)*(d_i + k_ue)
    f4 = -X*d + k_tl*(-R + R_tot)
    y = np.array([f0,f1,f2,f3,f4])
    return y



In [327]:
import numpy as np
from scipy.integrate import odeint
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import Legend
output_notebook()
# States : P, C1, T, R, C2, E, C3, X
# params_values = [30, 10, 0.5, 80, 2, 5, 10, 2, 1, 0.5, 20, 100, 400, 5]

list_of_seeds = [1e-3, 1e-2, 1e-1, 10, 100, 1e2]
vary_params = [0,1,2,3,4,5]

for pi in range(len(vary_params)):
    i = vary_params[pi]
    for seed in list_of_seeds:
        E_tot = 20
        P_tot = 100
        R_tot = 400
        k_bp = 30
        k_up = 10
        k_tx = 0.5
        k_br = 80
        k_ur = 2
        k_tl = 8
        params_values = [k_bp, k_up, k_tx, k_br, k_ur, k_tl, 10, 2, 1, 0.8, E_tot, P_tot, R_tot, 10]
        params_values[i] = params_values[i]*seed
        k_bp, k_up, k_tx, k_br, k_ur, k_tl,k_be, k_ue, d_i, d, E_tot, P_tot, R_tot, G = params_values
        timepoints = np.linspace(0,20,100)
        x_init_full = np.array([P_tot, 0, 0, R_tot, 0, E_tot, 0, 0])
        x_init1 = np.array([0,0])
        x_init2 = np.array([0,R_tot,0])
        x_init_cons = np.array([P_tot, 0, R_tot, E_tot, 0])
        y_full = odeint(lambda t,x: full_model(t,x,params_values), x_init_full, timepoints, tfirst = True)
        y1 = odeint(lambda t, x: model1(t,x,params_values), x_init1, timepoints, tfirst = True)
        y2 = odeint(lambda t, x: model2(t,x,params_values), x_init2, timepoints, tfirst = True)
        y_cons = odeint(lambda t, x: f_cons(t,x,params_values), x_init_cons, timepoints, tfirst = True)
        # Plotting
        from bokeh.io import output_notebook, show
        from bokeh.plotting import figure, show
        output_notebook()
        p = figure(plot_width=350, plot_height=250, title = 'Varying' + str(params_values[i]/seed) + ' by ' + str(seed))
        p.line(timepoints, y_full[:,-1], line_width = 2, color = 'black', legend = 'Full Model')
        p.line(timepoints, y1[:,-1], line_width = 2, color = 'red', legend = '[T, X]')
        p.line(timepoints, y2[:,-1], line_width = 2, color = 'blue', legend = '[T, R, X]')
        p.line(timepoints, y_cons[:,-1], line_width = 8, color = 'black', line_dash = [6,20], legend = 'Model w/ conservation')
        p.legend.location = 'bottom_right'
        show(p)

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [357]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import Legend
output_notebook()
list_of_seeds = [1e-3, 1e-2, 100, 1e2]
vary_params = [3,4,5]

for pi in range(len(vary_params)):
    i = vary_params[pi]
    for seed in list_of_seeds:
        E_tot = 20
        P_tot = 100
        R_tot = 400
        k_bp = 30
        k_up = 10
        k_tx = 0.5
        k_br = 80
        k_ur = 2
        k_tl = 8
        params_values = [k_bp, k_up, k_tx, k_br, k_ur, k_tl, 10, 2, 1, 0.8, E_tot, P_tot, R_tot, 10]
        params_values[i] = params_values[i]*seed
        k_bp, k_up, k_tx, k_br, k_ur, k_tl,k_be, k_ue, d_i, d, E_tot, P_tot, R_tot, G = params_values
        timepoints = np.linspace(0,50,100)
        x_init_full = np.array([P_tot, 0, 0, R_tot, 0, E_tot, 0, 0])
        x_init1 = np.array([0,0])
        x_init2 = np.array([0,R_tot,0])
        x_init_cons = np.array([P_tot, 0, R_tot, E_tot, 0])
        y_full = odeint(lambda t,x: full_model(t,x,params_values), x_init_full, timepoints, tfirst = True)
        y1 = odeint(lambda t, x: model1(t,x,params_values), x_init1, timepoints, tfirst = True)
        y2 = odeint(lambda t, x: model2(t,x,params_values), x_init2, timepoints, tfirst = True)
        y_cons = odeint(lambda t, x: f_cons(t,x,params_values), x_init_cons, timepoints, tfirst = True)
        # Plotting

        p = figure(plot_width=400, plot_height=320, title = 'Varying k_tl')
        full = p.line(timepoints, y_full[:,-1], line_width = 2, color = 'black')
        tx = p.line(timepoints, y1[:,-1], line_width = 2, color = 'red')
        trx = p.line(timepoints, y2[:,-1], line_width = 2, color = 'blue')
        fullw = p.line(timepoints, y_cons[:,-1], line_width = 8, color = 'black', line_dash = [6,20])
        legend = Legend(items=[
            ("Full Model", [full]),
            ("[T, X]", [tx]),
            ("[T, R, X]" , [trx]),
            ("Full model w/ conservation" , [fullw]),
        ], location="center")
        p.xaxis.axis_label = "Time"
        p.yaxis.axis_label = "[X]"
#         p.add_layout(legend, 'right')
        show(p)

Loading BokehJS ...

In [87]:
p.legend

[]